# Uitleg Notebook 
Dit Notebook is om de EDU files in te lezen en op te slaan als h5 file in GitHub

## EDU files content 
korte beschrijving wat er in die data staat

### plan van aanpak 

#### csv file exporten naar Github
* temporary map aanmaken
* Link naar Data invoeren
* download de zipfile en plaats in temporary map
* Unzippen in temporary map
* cvs file selecteren die er toe doet: voorbeeld naam: profielen elektriciteit 2021 versie 1.00 of Prognoseprofielen-elektriciteit-2022-2023-2024-dd-2020
* van csv file Dataframe maken
* geselecteerd csv file plaatsen op Github, map data-raw, in nieuwe map genaamd EDU
* temp folder leeggooien
* herhalen voor alle zip files (in totaal zijn er 7 (6 profielen en 1 prognoseprofiel)


#### Excell file omzetten naar DF






In [57]:
import os  
import requests
import zipfile as zp
from zipfile import ZipFile  # importeer van de module zipfile de Class ZipFile

In [58]:
# temporary variable aanmaken voor eventuele temp map
tmp_directory = "temp_voor_EDU_files"

#parent_dir = os.getcwd() # commando voor get Current Working Directory
current_dir = os.getcwd()

# ga naar temp directory toe

if tmp_directory not in current_dir:
    path = os.path.join(current_dir, tmp_directory) # extra variabele onder notebook (=cwd) wordt nu gemaakt
    # maak temp directory als hij nog niet bestaat
    try:
        os.mkdir(path)  # extra directory onder notebook (=cwd) wordt nu gemaakt
        print(f"Directory created {tmp_directory}") 
    except:
        print ('directory bestaat al')
    
    os.chdir(path)  # veranderen naar de temp directory
print(os.getcwd())


Directory created temp_voor_EDU_files
c:\Users\ericr\OneDrive - Actondata\Projecten\jads_enexis\notebooks\temp_voor_EDU_files


In [59]:
EDU_files = [
    'https://www.nedu.nl/wp-content/uploads/2020/07/Profielen-elektriciteit-2021-versie-1.00-Platform-Verbruiksprofielen-NEDU-20200701-002.1.1.zip',
    'https://www.nedu.nl/wp-content/uploads/2019/07/Profielen-elektriciteit-2020-versie-1.00-Platform-Verbruiksprofielen-NEDU-20190703-002.1.1.zip',
    'https://www.nedu.nl/wp-content/uploads/2018/07/Profielen-Elektriciteit-2019-versie-1.00-ALV-NEDU-20180704-011.1.1.zip',
    'http://www.nedu.nl/wp-content/uploads/2017/07/Profielen-Elektriciteit-2018-versie-1.00-ALV-NEDU-20170705-011.1.1.zip',
    'http://www.nedu.nl/wp-content/uploads/2016/04/Profielen-Elektriciteit-2017-versie-1.00-ALV-NEDU-20160413-012.1.1.zip',
    'http://www.nedu.nl/wp-content/uploads/2015/09/Profielen-Elektriciteit-2016-versie-1.01-ALV-NEDU-20150916-012.2.zip'
]

def download_url(url, save_path, chunk_size=128): # downloaden in chuncks, stream, LET OP: path = directory + filenaam
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:                    #wb = write binairy, text files gaan net iets anders (optimalisatie)
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

for EDU_file in EDU_files:
    # definieren van variabele van de download URL
    save_path = EDU_file.split("/")[-1]

    # Haal het bestand op en sla het op in de temporary directory
    download_url(EDU_file, save_path)

    # Creer een ZipFile Object met het pad naar de zip file
    with ZipFile(save_path, 'r') as zipObj:
        # Extract de zip file naar de huidige directory
        zipObj.extractall(path=None, members=None, pwd=None)


In [76]:
import glob
from glob import glob

import pandas as pd

combined_df = None
files = glob('profielen*.csv')  # resultaat = lijst met bestaandsnamen komt hier uit. Als er 1 bestandsnaam in zit is dit nog steeds een lijst

# itereer door alle csv bestanden met het patroon. 
for file in files:
    # Haal de data uit de csv file. De eerste 4 rijen bevatten alleen een beschrijving en geen data
    df = pd.read_csv(file)
    df = df.drop([0,1,2,3])

    # De namen van de kolommen
    df.columns = ('Meting', 'Start','End','E1A','E1B','E1C','E2A','E2B','E3A','E3B','E3C','E3D','E4A')

    # De eerste kolom bevat het tijdstip van de meting. Deze hebben we ook niet nodig
    df = df.drop(['Meting'], axis=1)

    # Reindex het dataframe zodat het bij index 0 begint
    df = df.reset_index(drop=True)

    if combined_df is None:
        combined_df = df.copy()
    else:
        combined_df = combined_df.append(df)

# Pas de kolom types aan
combined_df = combined_df.astype({'Start':'datetime64','End':'datetime64','E1A':'float64','E1B':'float64','E1C':'float64','E2A':'float64','E2B':'float64','E3A':'float64','E3B':'float64','E3C':'float64','E3D':'float64','E4A':'float64'})

# Het dataframe hoeft niet gevuld te worden op volgorde van tijd. Het wordt gevuld op volgorde van het
# vinden van de bestanden door glob. Sorteer het daarom op de tijden in de start kolom
combined_df = combined_df.sort_values(by='Start')

# Reindex het hele dataframe
combined_df = combined_df.reset_index(drop=True)

combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175392 entries, 0 to 175391
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   Start   175392 non-null  datetime64[ns]
 1   End     175392 non-null  datetime64[ns]
 2   E1A     175392 non-null  float64       
 3   E1B     175392 non-null  float64       
 4   E1C     175392 non-null  float64       
 5   E2A     175392 non-null  float64       
 6   E2B     175392 non-null  float64       
 7   E3A     175392 non-null  float64       
 8   E3B     175392 non-null  float64       
 9   E3C     175392 non-null  float64       
 10  E3D     175392 non-null  float64       
 11  E4A     175392 non-null  float64       
dtypes: datetime64[ns](2), float64(10)
memory usage: 16.1 MB


In [77]:
combined_df.head(15)

,Start,End,E1A,E1B,E1C,E2A,E2B,E3A,E3B,E3C,E3D,E4A
0,2016-01-01 00:00:00,2016-01-01 00:15:00,0.000040,0.000041,0.000037,0.000023,0.000025,0.000020,0.000020,0.000020,0.000025,0.00006
1,2016-01-01 00:15:00,2016-01-01 00:30:00,0.000039,0.000039,0.000035,0.000022,0.000025,0.000020,0.000021,0.000021,0.000025,0.00006
2,2016-01-01 00:30:00,2016-01-01 00:45:00,0.000037,0.000038,0.000034,0.000022,0.000025,0.000020,0.000021,0.000021,0.000025,0.00006
3,2016-01-01 00:45:00,2016-01-01 01:00:00,0.000036,0.000037,0.000036,0.000022,0.000025,0.000020,0.000021,0.000021,0.000025,0.00006
4,2016-01-01 01:00:00,2016-01-01 01:15:00,0.000034,0.000035,0.000037,0.000023,0.000025,0.000020,0.000021,0.000021,0.000025,0.00006
5,2016-01-01 01:15:00,2016-01-01 01:30:00,0.000032,0.000034,0.000038,0.000023,0.000024,0.000020,0.000021,0.000021,0.000025,0.00006
6,2016-01-01 01:30:00,2016-01-01 01:45:00,0.000030,0.000033,0.000038,0.000022,0.000024,0.000020,0.000021,0.000021,0.000025,0.00006
7,2016-01-01 01:45:00,2016-01-01 02:00:00,0.000027,0.000032,0.000037,0.000021,0.000023,0.000020,0.000021,0.000021,0.000025,0.00006
8,2016-01-01 02:00:00,2016-01-01 02:15:00,0.000026,0.000032,0.000034,0.000020,0.000022,0.000020,0.000021,0.000021,0.000025,0.00006
9,2016-01-01 02:15:00,2016-01-01 02:30:00,0.000024,0.000031,0.000032,0.000020,0.000022,0.000020,0.000021,0.000021,0.000025,0.00006


In [79]:
# Bereken de index van de jaarovergang van 2016 naar 2017 om te controleren of de sortering heeft gewerkt
end_of_year = 4 * 24 * 366
combined_df[end_of_year-10:end_of_year+10]

,Start,End,E1A,E1B,E1C,E2A,E2B,E3A,E3B,E3C,E3D,E4A
35126,2016-12-31 21:30:00,2016-12-31 21:45:00,0.000049,0.000046,0.000050,0.000029,0.000024,0.000020,0.000022,0.000022,0.000027,0.00006
35127,2016-12-31 21:45:00,2016-12-31 22:00:00,0.000049,0.000045,0.000051,0.000027,0.000024,0.000020,0.000022,0.000022,0.000027,0.00006
35128,2016-12-31 22:00:00,2016-12-31 22:15:00,0.000048,0.000044,0.000051,0.000027,0.000024,0.000020,0.000021,0.000021,0.000026,0.00006
35129,2016-12-31 22:15:00,2016-12-31 22:30:00,0.000046,0.000044,0.000049,0.000026,0.000023,0.000019,0.000021,0.000021,0.000026,0.00006
35130,2016-12-31 22:30:00,2016-12-31 22:45:00,0.000044,0.000044,0.000046,0.000026,0.000023,0.000020,0.000021,0.000021,0.000026,0.00006
35131,2016-12-31 22:45:00,2016-12-31 23:00:00,0.000042,0.000044,0.000043,0.000025,0.000022,0.000020,0.000021,0.000021,0.000026,0.00006
35132,2016-12-31 23:00:00,2016-12-31 23:15:00,0.000041,0.000042,0.000039,0.000026,0.000023,0.000020,0.000021,0.000021,0.000026,0.00006
35133,2016-12-31 23:15:00,2016-12-31 23:30:00,0.000041,0.000041,0.000039,0.000027,0.000023,0.000019,0.000021,0.000021,0.000026,0.00006
35134,2016-12-31 23:30:00,2016-12-31 23:45:00,0.000040,0.000041,0.000038,0.000028,0.000023,0.000019,0.000020,0.000020,0.000026,0.00006
35135,2016-12-31 23:45:00,2017-01-01 00:00:00,0.000040,0.000040,0.000037,0.000028,0.000023,0.000019,0.000020,0.000020,0.000025,0.00006


In [80]:
# data locatie als we hier komen is 'temp_voor_EDU_files'
data_location = '../../data/processed'

# Ga naar de folder waarin de processed data staat
# Eenmaal chdir uitgevoerd blijft de current directory uiteraard 'processed'
if 'processed' not in os.getcwd():
    os.chdir(data_location)

In [81]:
try:
    os.remove('nedu_files.h5')
except:
    print ('File nog in gebruik of niet gevonden')

store = pd.HDFStore('nedu_files.h5')
store['nedu_files'] = combined_df
store.close()

In [82]:
# Gooi de temporary directory weg. Het dataframe is gemaakt en opgeslagen
data_location = '../../notebooks'

# Ga terug naar de notebooks directory
if 'notebooks' not in os.getcwd():
    os.chdir(data_location)

In [83]:
import shutil

try:
    # Gooi de tijdelijke directory weg. Het dataframe is gemaakt en opgeslagen
    shutil.rmtree(tmp_directory)
except OSError as e:
    print(f"Error: {tmp_directory} : {e.strerror}")